<a href="https://colab.research.google.com/github/Hadia-git-sketch/ArchTechnologies_1st_MonthTasks/blob/main/MNIST_Digit_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models
import gradio as gr
import numpy as np
import cv2
from scipy import ndimage

# --- 1. BUILD & TRAIN MODEL ---
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = models.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5, verbose=1)

# --- 2. ADVANCED PREPROCESSING  ---
def get_best_shift(img):
    cy, cx = ndimage.center_of_mass(img)
    rows, cols = img.shape
    shiftx = np.round(cols/2.0 - cx).astype(int)
    shifty = np.round(rows/2.0 - cy).astype(int)
    return shiftx, shifty

def shift(img, sx, sy):
    rows, cols = img.shape
    M = np.float32([[1, 0, sx], [0, 1, sy]])
    shifted = cv2.warpAffine(img, M, (cols, rows))
    return shifted

def classify_digit(input_data):
    if input_data is None: return "Draw a digit"

    # 1. Get grayscale image
    img = input_data["composite"]
    gray = cv2.cvtColor(img, cv2.COLOR_RGBA2GRAY)

    # 2. Resize to 28x28 and apply threshold
    resized = cv2.resize(gray, (28, 28), interpolation=cv2.INTER_AREA)
    _, thresh = cv2.threshold(resized, 30, 255, cv2.THRESH_BINARY)

    # 3. Center the digit (Crucial for MNIST accuracy)
    try:
        sx, sy = get_best_shift(thresh)
        final_img = shift(thresh, sx, sy)
    except:
        final_img = thresh

    # 4. Predict
    prediction = model.predict(final_img.reshape(1, 28, 28) / 255.0, verbose=0)[0]
    return {str(i): float(prediction[i]) for i in range(10)}

# --- 3. INTERFACE ---
demo = gr.Interface(
    fn=classify_digit,
    inputs=gr.Sketchpad(label="Draw Here", type="numpy"),
    outputs=gr.Label(num_top_classes=3),
    title="Arch Technologies: DIGIT RECOGNITION"
)

if __name__ == "__main__":
    demo.launch()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - accuracy: 0.8914 - loss: 0.3633
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 17s 9ms/step - accuracy: 0.9716 - loss: 0.0996
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 18s 10ms/step - accuracy: 0.9799 - loss: 0.0648
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - accuracy: 0.9846 - loss: 0.0476
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 20s 9ms/step - accuracy: 0.9882 - loss: 0.0384
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5e4977c65bd3eb7bdb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/s